In [1]:
from sub_func import *
from pf_selection import *
from pipeline_utils import *

from t1 import *
from t2_t4 import *
from t5 import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


예시로, 2023년 1분기 분석을 위해 2022년 4분기 정보를 바탕으로 2023년 1분기 초기 포트폴리오 생성

In [2]:
# 조회할 정보의 기간
base_year = '2022'
base_quarter = 'Q4'
start_date = '20221001'
end_date = '20221231'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q1'

# 'pf_selection_agent'에 초기 포트폴리오가 있는지 확인
db_info = get_all_page_ids_from_database('pf_selection_agent')
init_pf_page_title = f"{target_year}_{target_quarter}_init_pf"

if init_pf_page_title not in list(db_info.keys()):
    print('초기 포트폴리오를 생성합니다...')
    pf_selection_info = pf_selection_main(base_year, base_quarter, start_date, end_date)
    print("노션에 output을 모두 저장합니다...")
    to_DB('pf_selection_agent', init_pf_page_title, f"{start_date}_{end_date}", str(pf_selection_info))

# 'pf_selection_agent'에서 종목 코드 가져오기
tickers = get_tickers_from_json('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')
weight_dict = {ticker: 1 / len(tickers) for ticker in tickers}
 
# JSON 파일 저장할 디렉토리 경로
log_dir = f"./pf_logs/{target_year}_{target_quarter}"
os.makedirs(log_dir, exist_ok=True)  # 디렉토리 없으면 생성

# 파일 경로
file_path = os.path.join(log_dir, f"{target_year}_{target_quarter}_init_pf.json")

with open(f'{file_path}', "w", encoding="utf-8") as f:
    json.dump(weight_dict, f, indent=2, ensure_ascii=False)

초기 포트폴리오를 생성합니다...
2022의 20221001 ~ 20221231 섹터 선정 보고서 생성 시작...
--------------------------------------------------
국제 뉴스 수집 완료!
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'
--------------------------------------------------
거시경제 정보 수집 완료!
--------------------------------------------------
인덱스 가격 지표 수집 완료! GPT 생성을 기다리는 중...
--------------------------------------------------
GPT 응답 수신 완료!
최종 포트폴리오 종목 선정 시작...
--------------------------------------------------
포트폴리오 구성 시작...
--------------------------------------------------
(1/792)
61439
--------------------------------------------------
(2/792)
83073
--------------------------------------------------
(3/792)
14735
--------------------------------------------------
(4/792)
69710
--------------------------------------------------
(5/792)
23730
------------------------------

In [3]:
# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
kospi_open_days = get_kospi_open_days("20230101", "20230331")

In [4]:
def finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date):

    print('***** t1_analyst_main 실행 *****')
    t1_analyst_main(today, tickers, base_year, base_quarter)

    print('***** t1_pf_manager_main 실행 *****')
    t1_pf_manager_main(today, tickers, base_year, base_quarter, target_year, target_quarter)

    print('***** t1_trader_main 실행 *****')
    t1_trader_main(today, tickers, base_year, base_quarter)

    print('***** t2_t4_main 실행 *****')
    t_2_t_4_main(tickers, start_date, today, target_year, target_quarter, base_year, base_quarter)

    print('***** t5_main 실행 *****')
    t_5_main(today, base_year, base_quarter)

In [5]:
test_span = kospi_open_days[:10]

# 4,401,782
for today in test_span:
    finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date)
    print(f'{today}일자 실행 완료')

***** t1_analyst_main 실행 *****
[t1_analyst] === 종목별 분석 시작 ===
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'

=== 025820 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/025820'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/025820'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/025820'
025820의 이전 년도 재무 데이터를 불러올 수 없습니다.
025820의 fin_statement_info 정보를 확인할 수 없습니다.
analyze_financial_data의 GPT 요청 결과: {'id': 'ch

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 000040 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/000040/_000040_재무제표 ().csv'
000040의 이전 년도 재무 데이터를 불러올 수 없습니다.
000040의 fin_statement_info 정보를 확인할 수 없습니다.
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxFh9AcrAcN6Uq8EYsCXOGtJnajMU', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 분석 보고서\n\n## 1. 요약 재무 정보\n\n### 1.1 재무상태표(2022년 12월 기준)\n\n#### 자산\n- **유동자산**: 60,741억원 (2021년: 69,663억원, 2020년: 60,971억원)\n- **비유동자산**: 88,444억원 (2021년: 98,774억원, 2020년: 94,903억원)\n- **자산 총계**: 149,185억원 (2021년: 168,437억원, 2020년: 155,873억원)\n\n#### 부채\n- **유동부채**: 93,561억원 (2021년: 95,522억원, 2020년: 84,123억원)\n- **비유동부채**: 15,600억원 (2021년: 22,689억원, 2020년: 21,905억원)\n- **부채 총계**: 109,161억원 (2021년: 118,211억원, 2020년: 106,028억원)\n\n#### 자본\n- **자본 총계**: 40,024억원 (2021년: 55,226억원, 2020년: 49,845억원)\n\n#

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 011700 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxFkbjL5V5iTq6OX4wz0GhQa5vmbO', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 분석 보고서\n\n## 1. 재무상태표 요약\n- **유동자산**: 2022년 73,783억원, 2021년 69,346억원, 2020년 64,740억원으로 지속적으로 증가.\n- **비유동자산**: 2022년 41,939억원, 2021년 31,803억원, 2020년 30,473억원으로 역시 증가.\n- **자산총계**: 115,722억원으로 전년 대비 증가.\n- **유동부채**: 21,937억원으로 전년 대비 증가.\n- **부채총계**: 22,776억원으로 전년 대비 증가.\n- **자본총계**: 92,812억원으로 전년 대비 증가.\n\n## 2. 손익계산서 요약\n- **매출액**: 2022년 56,040억원, 2021년 54,365억원으로 증가.\n- **영업이익**: 2022년 -4,096억원, 2021년 5,161억원으로 적자 전환.\n- **당기순이익**: 2022년 -4,734억원, 2021년 4,087억원으로 적자 전환.\n\n## 3. 주요 재무 비율\n- **주가 (Stock Price)**: 6,400원\n- **PER (주가수익비율)**: -43.87 (적자)\n- **PBR (주가순자산비율)**: 2.23\n- **ROE (자기자본이익률)**: -5.09% (적자)\n- **Profit Growth Rate**: -215.80% (성장률 감소)\n- **CAGR (연평균 성장률)**: -2.16% (성장률 감소)\n\n## 4. 분석 및 의견\n### 4.1. 자산 및 부채\n회사의 자산은 지속적으로 증가하고 있지만, 유동부채도 증가하고 있어 단기적인 유동성 위험이 우려

...

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 249420 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxMhwNuP7OCr4DJ6Rf6Vr74hv2vXd', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 재무제표 요약\n\n### 1.1. 연결 재무상태표 (2022년 12월 31일 기준)\n- **자산총계**: 655,810억원 (전년 대비 감소: 7.5%)\n- **유동자산**: 282,035억원 (전년 대비 감소: 6.5%)\n- **비유동자산**: 373,775억원 (전년 대비 감소: 8.3%)\n- **부채총계**: 457,586억원 (전년 대비 감소: 16.5%)\n- **자본총계**: 198,224억원 (전년 대비 증가: 23.4%)\n\n### 1.2. 연결 포괄손익계산서 (2022년 1월 1일 ~ 2022년 12월 31일)\n- **매출액**: 637,714억원 (전년 대비 증가: 13.8%)\n- **영업이익**: △73,481억원 (전년 대비 손실 확대)\n- **당기순이익**: △142,170억원 (전년 대비 손실 확대)\n\n### 1.3. 주요 재무 비율\n- **주가**: 28,550원\n- **PER (주가수익비율)**: -5.38 (손실)\n- **PBR (주가순자산비율)**: 3.86\n- **ROE (자기자본이익률)**: -71.72%\n- **이익 성장률**: 40.81%\n- **CAGR**: 40.81%\n\n## 2. 재무제표 분석\n\n### 2.1. 자산 분석\n- **유동자산**이 감소하였으며, 특히 **현금 및 현금성 자산**은 52,146억원에서 135,586억원으로 급감하였다. 이는 기업의 유동성 문제를 나타낼 수 있음.\n- **비유동자산** 역시 감소했으나, **유형자산**은 소폭 증가하였고, **무형자산**은 크게 줄

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 008770 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxMjQjEOuSecvlUa04Om9pcCDmGjm', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 주요 재무 제표 요약\n\n### 1.1 재무상태표 요약\n\n- **유동자산**: 1,478,831 백만원 (제50기), 이전 기수 대비 증가\n- **비유동자산**: 1,459,714 백만원 (제50기), 이전 기수 대비 감소\n- **자산총계**: 2,938,545 백만원 (제50기), 이전 기수 대비 증가\n\n- **유동부채**: 1,305,979 백만원 (제50기), 이전 기수 대비 증가\n- **비유동부채**: 1,092,782 백만원 (제50기), 이전 기수 대비 증가\n- **부채총계**: 2,398,761 백만원 (제50기), 이전 기수 대비 증가\n\n- **자본총계**: 539,784 백만원 (제50기), 이전 기수 대비 감소\n\n### 1.2 손익계산서 요약\n\n- **매출액**: 4,922,009 백만원 (제50기), 이전 기수 대비 증가\n- **영업이익**: 78,346 백만원 (제50기), 이전 기수 대비 감소\n- **당기순이익**: (50,164) 백만원 (제50기), 이전 기수 대비 적자 확대\n\n## 2. 주요 재무 비율 분석\n\n- **주가**: 83,100 원\n- **PER**: -65.02 (적자)\n- **PBR**: 6.04 (상대적으로 높은 수준)\n- **ROE**: -9.29% (자본 대비 수익성 저하)\n- **이익 성장률**: -285.35% (매우 부정적인 성장세)\n- **CAGR**: -2.85% (장기적인 성장세 둔화)\n\n## 3. 분석 및 의견\n\n### 3.1 유동성 및 안정성\n- **유동자

KeyboardInterrupt: 

t1 분석 실행

***** t1_analyst_main 실행 *****
[t1_analyst] === 종목별 분석 시작 ===
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'

=== 020000 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/020000/_020000_재무제표 ().csv'
020000의 이전 년도 재무 데이터를 불러올 수 없습니다.
020000의 fin_statement_info 정보를 확인할 수 없습니다.
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ax3IOsDqoqDMEzUnRxpB5ZWkKErWa', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# **재무제표 분석 보고서**\n\n## 1. 개요\n본 보고서는 주식회사 한섬의 2022년 제36기 재무제표를 기반으로 작성되었습니다. 주요 재무정보를 분석하고, 기업의 재무 건전성과 주식 투자 관점에서의 투자 가치를 평가합니다.\n\n## 2. 재무제표 요약\n\n### 2.1. 자산\n- **총 자산**: 1,684,715,498,594 원\n  - 유동자산: 1,031,083,371,680 원 (전년 대비 증

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 031430 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ax3JknuVtMoiPvwUnsG2xqigeYFei', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무보고서 분석 및 투자 의견\n\n## 1. 재무제표 요약\n### 1.1. 자산\n- **유동자산**: 2022년 547,064,855,331원, 2021년 459,823,817,817원, 2020년 426,786,183,462원으로 지속적인 증가세 보임.\n- **비유동자산**: 2022년 731,616,959,303원, 2021년 699,303,760,568원, 2020년 720,668,262,757원으로 소폭의 변동을 보임.\n- **자산총계**: 1,278,681,814,634원으로 증가세.\n\n### 1.2. 부채\n- **유동부채**: 282,263,660,680원으로 증가.\n- **비유동부채**: 162,742,126,293원으로 감소.\n- **부채총계**: 445,005,786,973원으로 증가.\n\n### 1.3. 자본\n- **지배기업의 소유주 귀속자본**: 830,517,772,720원으로 증가.\n- **자본총계**: 833,676,027,661원으로 증가.\n\n### 1.4. 손익계산서\n- **매출액**: 1,553,877,316,519원으로 증가.\n- **영업이익**: 115,270,511,523원으로 증가.\n- **당기순이익**: 118,792,238,428원으로 증가.\n\n## 2. 주요 재무 비율\n- **주가**: 24,750원\n- **PER**: 7.44 (저평가 가능성)\n- **PBR**: 1.06 (자산 대비 적정한 수준)\n- **ROE**: 14.25% (우수한 수익성)\n- **Profit Growth Rate**: 43.77% (높은 성장

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 093050 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Ax3KU5m6aYVwNAl6YQUaRwfRFknKx', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 요약 재무 정보\n\n### 1.1 주요 재무 비율\n- **주가 (Stock Price)**: 15,900 KRW\n- **PER (주가수익비율)**: 2.62\n- **PBR (주가순자산비율)**: 0.28\n- **ROE (자기자본이익률)**: 10.70%\n- **이익 성장률 (Profit Growth Rate)**: 30.22%\n- **CAGR (연평균 성장률)**: 30.22%\n\n### 1.2 요약 연결 재무 상태표 (2022년 기준)\n- **자산 총계**: 2,584,040 백만 원\n- **비유동자산**: 1,238,270 백만 원\n- **유동자산**: 1,345,770 백만 원\n- **부채 총계**: 927,192 백만 원\n- **자본 총계**: 1,656,848 백만 원\n\n### 1.3 요약 손익계산서 (2022년 기준)\n- **매출액**: 1,968,540 백만 원\n- **영업이익**: 185,161 백만 원\n- **당기순이익**: 177,303 백만 원\n- **기본주당이익**: 5,351 원\n\n## 2. 재무 분석\n\n### 2.1 자산 및 부채 분석\n- **자산의 증가**: 2022년 자산 총계는 2,584,040 백만 원으로 전년 대비 증가하였으며, 이는 비유동자산의 증가(1,238,270 백만 원)에서 주로 기인한 것으로 보입니다.\n- **부채 감소**: 부채 총계는 927,192 백만 원으로, 이전 연도의 1,042,768 백만 원에서 감소하였습니다. 이는 재무 건전성의 개선을 나타냅니다.\n\n### 2.2 수

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.1936 - mae: 0.6093 - val_loss: 0.0208 - val_mae: 0.1585 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1543 - mae: 0.5430 - val_loss: 0.0138 - val_mae: 0.1356 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1231 - mae: 0.4793 - val_loss: 0.0091 - val_mae: 0.1221 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0880 - mae: 0.4033 - val_loss: 0.0067 - val_mae: 0.1074 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0653 - mae: 0.3384 - val_loss: 0.0067 - val_mae: 0.0992 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0447 - mae: 0.2731 - val_loss: 0.0086 - val_mae: 0.1065 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0297 - mae: 0.2177 - val_loss: 0.0127 - val_mae: 0.1278 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/s

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.1240 - mae: 0.4890 - val_loss: 0.0444 - val_mae: 0.2688 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0786 - mae: 0.3871 - val_loss: 0.0298 - val_mae: 0.2290 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0539 - mae: 0.3110 - val_loss: 0.0227 - val_mae: 0.1945 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0268 - mae: 0.2082 - val_loss: 0.0236 - val_mae: 0.1796 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0110 - mae: 0.1164 - val_loss: 0.0321 - val_mae: 0.1816 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0050 - mae: 0.0745 - val_loss: 0.0453 - val_mae: 0.2260 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0063 - mae: 0.0974 - val_loss: 0.0571 - val_mae: 0.2749 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/s

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0816 - mae: 0.3488 - val_loss: 0.1463 - val_mae: 0.5132 - learning_rate: 0.0010
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0638 - mae: 0.2924 - val_loss: 0.1061 - val_mae: 0.4275 - learning_rate: 0.0010
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0423 - mae: 0.2321 - val_loss: 0.0731 - val_mae: 0.3416 - learning_rate: 0.0010
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0251 - mae: 0.1674 - val_loss: 0.0473 - val_mae: 0.2655 - learning_rate: 0.0010
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0211 - mae: 0.1580 - val_loss: 0.0287 - val_mae: 0.1997 - learning_rate: 0.0010
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0112 - mae: 0.1191 - val_loss: 0.0185 - val_mae: 0.1483 - learning_rate: 0.0010
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0083 - mae: 0.1053 - val_loss: 0.0158 - val_mae: 0.1459 - learning_rate: 0.0010
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/s

t2 ~ t4 분석 실행

=== 1/22 ===
=== 2/22 ===
=== 3/22 ===
=== 4/22 ===
=== 5/22 ===
=== 6/22 ===
'Timestamp' | Error occurred at 300720
=== 7/22 ===
=== 8/22 ===
=== 9/22 ===
=== 10/22 ===
'Timestamp' | Error occurred at 000990
=== 11/22 ===
=== 12/22 ===
=== 13/22 ===
=== 14/22 ===
=== 15/22 ===
=== 16/22 ===
'Timestamp' | Error occurred at 000670
=== 17/22 ===
=== 18/22 ===
=== 19/22 ===
=== 20/22 ===
=== 21/22 ===
=== 22/22 ===
=== [9] 1/22 (020000) ===
=== [9] 2/22 (031430) ===
=== [9] 3/22 (093050) ===
=== [9] 4/22 (004370) ===
=== [9] 5/22 (011420) ===
=== [9] 6/22 (300720) ===
=== [9] 7/22 (005820) ===
=== [9] 8/22 (195870) ===
=== [9] 9/22 (077500) ===
=== [9] 10/22 (000990) ===
=== [9] 11/22 (003160) ===
=== [9] 12/22 (029460) ===
=== [9] 13/22 (007810) ===
=== [9] 14/22 (092220) ===
=== [9] 15/22 (025560) ===
=== [9] 16/22 (000670) ===
=== [9] 17/22 (088790) ===
=== [9] 18/22 (001820) ===
=== [9] 19/22 (120030) ===
=== [9] 20/22 (036570) ===
=== [9] 21/22 (000210) ===
=== [9] 22/22 (009970) ===

NameError: name 't_5_main' is not defined